In [1]:
import sys
sys.path.append('../')
import wd

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:

wdl_model = wd.WideAndDeepModel([100 for i in range(22)],8,[400,400,400],0.1)
traced = symbolic_trace(wdl_model)

In [9]:
def gen_pattern_replace_and_matcher_for_MLP(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = nn.Parameter(torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64)),False)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = x + self.offsets
          x = self.embed(x).view(-1,self.embed_output_dim)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      return self.redency_linear(self.embed(redency_part).view(-1,self.redency_weight_len)) + self.unredency_linear(self.embed(unredency_part).view(-1,self.unredency_weight_len))
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def gen_pattern_replace_and_matcher_for_MLP2(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = x + self.offsets
          x = self.embed(x).view(-1,self.embed_output_dim)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.offsets =  torch.as_tensor(np.array(offsets_val, dtype=np.int64))
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0])
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      x = self.embed(x + self.offsets)
      return self.redency_linear(x[redency_part_slice].view(-1,self.redency_weight_len)) + self.unredency_linear(x[unredency_part_slice].view(-1,self.unredency_weight_len))
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [6]:

wdl_model = wd.WideAndDeepModel([100 for i in range(22)],128,[400,400,400],0.1)
traced = symbolic_trace(wdl_model)

In [7]:
def workload_wdl(dim,l = [400,400,400]):
  print(f"now gen workload of wdl with config: dim: {dim}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(22)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(22)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_MLP(modify_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

# l = [400,400,400]

# dim = 8

In [257]:
ori, modify = workload_wdl(8)

now gen workload of wdl with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [260]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.71 ms ± 544 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [263]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

8.87 ms ± 2.16 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [265]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

4.6 ms ± 1.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [267]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

12.2 ms ± 2.57 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32

In [268]:
ori, modify = workload_wdl(32)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [269]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.27 ms ± 883 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [271]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.5 ms ± 925 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [279]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

5.9 ms ± 1.04 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [280]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

5.68 ms ± 1.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64

In [10]:
ori, modify = workload_wdl(64)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [13]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.43 ms ± 720 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [14]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.34 ms ± 718 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [287]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

7.02 ms ± 1.24 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [289]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

6.18 ms ± 1.72 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128

In [640]:
ori, modify = workload_wdl(128)

now gen workload of wdl with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [641]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.12 ms ± 936 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [642]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.74 ms ± 912 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [643]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

13.9 ms ± 1.98 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [644]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

6.89 ms ± 1.28 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# torchscript

In [299]:
def workload_wdl_script(dim,l = [400,400,400]):
  print(f"now gen workload of wdl with config: dim: {dim}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(22)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(22)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_MLP(modify_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  ori = torch.jit.script(ori_traced)
  modify = torch.jit.script(wdl_model_modify)
  ori.eval()
  modify.eval()
  return ori,modify

# dim = 8

In [430]:
ori, modify = workload_wdl_script(8)

now gen workload of wdl with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [432]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.12 ms ± 428 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [433]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.12 ms ± 439 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [461]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

4.32 ms ± 1.27 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [464]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

4.19 ms ± 740 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32

In [309]:
ori, modify = workload_wdl_script(32)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [311]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.31 ms ± 481 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [313]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.29 ms ± 474 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [318]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

5.38 ms ± 1.77 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [319]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

5.16 ms ± 1.03 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64

In [466]:
ori, modify = workload_wdl_script(64)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [469]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.73 ms ± 622 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [468]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

2.77 ms ± 607 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [470]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

6.48 ms ± 1.65 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [471]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

6.43 ms ± 1.25 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128

In [628]:
ori, modify = workload_wdl_script(128)

now gen workload of wdl with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [630]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.56 ms ± 910 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [632]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.58 ms ± 691 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [639]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

12.9 ms ± 1.94 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [637]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

12.9 ms ± 1.75 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# l = [1024,512,256]

# dim = 8

In [361]:
ori, modify = workload_wdl(8,[1024,512,256])

now gen workload of wdl with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [363]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.68 ms ± 683 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [365]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

7.45 ms ± 2.19 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [366]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

15.8 ms ± 2.26 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [368]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

18.6 ms ± 2.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32

In [369]:
ori, modify = workload_wdl(32,[1024,512,256])

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [371]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.02 ms ± 717 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [373]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.8 ms ± 714 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [375]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

17 ms ± 1.51 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [377]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

16.7 ms ± 2.45 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64

In [378]:
ori, modify = workload_wdl(64,[1024,512,256])

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [384]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.58 ms ± 871 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [382]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.11 ms ± 828 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [386]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

18.3 ms ± 2.99 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [390]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

21.9 ms ± 2.29 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128

In [391]:
ori, modify = workload_wdl(128,[1024,512,256])

now gen workload of wdl with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [393]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

6.41 ms ± 1.25 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [395]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

5.19 ms ± 861 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [397]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

28.9 ms ± 2.89 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [399]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

21.1 ms ± 2.54 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# custom_workload 100

In [15]:
def custom_workload_wdl(dim,nums_fields,prefix,l = [400,400,400]):
  print(f"now gen workload of wdl with config: dim: {dim}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(nums_fields)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(nums_fields)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_MLP(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

# dim = 8, 40

In [16]:
ori,modify = custom_workload_wdl(8,100,40)

now gen workload of wdl with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [484]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

4.07 ms ± 784 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [485]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

4.84 ms ± 762 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [486]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

10.5 ms ± 1.37 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [487]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

10.6 ms ± 1.56 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32, 40

In [35]:
ori,modify = custom_workload_wdl(32,100,40)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [36]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

5.83 ms ± 1.05 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [38]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

5.41 ms ± 1.02 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [39]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

21.4 ms ± 3.88 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [40]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

11.7 ms ± 2.14 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [41]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 24.42026138305664 ms
total time: 24.78790283203125 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    mlp_mlp_0                         6.31738             25.4857
call_module    embedding_embedding               5.7826              23.3283
call_module    mlp_mlp_2                         1.98817              8.02074
call_module    mlp_mlp_6                         1.87969              7.5831
call_module    mlp_mlp_3                         1.83821              7.41574
call_module    mlp_mlp_8                         1.13082              4.56198
call_module    mlp_mlp_1                         1.10626              4.46291
call_module    mlp_mlp_4                         1.02139              4.1205
call_module    linear_fc                         0.836134             3.37315
call_function  add                               0.722647             2.

In [42]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 15.438079833984375 ms
total time: 15.74087142944336 ms
Op type        Op                  Average runtime (ms)    Pct total runtime
-------------  ----------------  ----------------------  -------------------
call_module    unredency_linear               5.65958             35.9547
call_module    mlp_mlp_6                      1.40572              8.93036
call_module    mlp_mlp_2                      1.38831              8.81979
call_module    mlp_mlp_3                      1.37973              8.76526
call_module    embed_1                        0.806332             5.12253
call_module    mlp_mlp_5                      0.690699             4.38793
call_module    linear_fc                      0.607967             3.86235
call_module    mlp_mlp_8                      0.51713              3.28527
call_module    mlp_mlp_7                      0.50807              3.22771
call_function  add                            0.375271             2.38405
call_function  add_3      

# dim = 64, 40

In [43]:
ori,modify = custom_workload_wdl(64,100,40)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [44]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

6.68 ms ± 2.45 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [45]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

5.81 ms ± 1.7 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [46]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

31.7 ms ± 3.48 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [47]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

22.7 ms ± 2.58 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [51]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 26.803255081176758 ms
total time: 27.068376541137695 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding              13.222               48.8466
call_module    mlp_mlp_0                         8.39233             31.0042
call_module    mlp_mlp_3                         0.895977             3.31005
call_module    mlp_mlp_6                         0.84281              3.11363
call_module    linear_fc                         0.511408             1.88932
call_module    mlp_mlp_2                         0.427961             1.58104
call_function  add                               0.387192             1.43042
call_module    mlp_mlp_8                         0.369072             1.36348
call_module    mlp_mlp_5                         0.365734             1.35115
call_module    mlp_mlp_1                         0.24271            

In [50]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 21.889686584472656 ms
total time: 22.223234176635742 ms
Op type        Op                  Average runtime (ms)    Pct total runtime
-------------  ----------------  ----------------------  -------------------
call_module    embed_1                        8.32009             37.4387
call_module    unredency_linear               7.00045             31.5006
call_module    mlp_mlp_3                      1.06359              4.78592
call_module    mlp_mlp_6                      1.00708              4.53165
call_module    linear_fc                      0.609636             2.74324
call_module    mlp_mlp_2                      0.547886             2.46537
call_module    mlp_mlp_5                      0.433207             1.94934
call_function  add                            0.401258             1.80558
call_module    mlp_mlp_8                      0.362635             1.63178
call_module    mlp_mlp_4                      0.287771             1.29491
call_module    mlp_mlp_7  

# dim = 128, 40

In [52]:
ori,modify = custom_workload_wdl(128,100,40)

now gen workload of wdl with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [508]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

16.4 ms ± 3.22 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [509]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

7.12 ms ± 1.56 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [510]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

49.9 ms ± 2.95 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [511]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

33.8 ms ± 2.52 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [53]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 42.16194152832031 ms
total time: 42.474985122680664 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    embedding_embedding              19.5169              45.9493
call_module    mlp_mlp_0                        15.9841              37.6317
call_module    mlp_mlp_8                         1.08004              2.54276
call_module    mlp_mlp_3                         0.96035              2.26098
call_module    mlp_mlp_6                         0.896931             2.11167
call_module    mlp_mlp_7                         0.598431             1.4089
call_module    linear_fc                         0.567675             1.33649
call_module    mlp_mlp_2                         0.478268             1.126
call_module    mlp_mlp_5                         0.37837              0.890807
call_function  add                               0.358582             0

In [54]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 29.746532440185547 ms
total time: 30.073881149291992 ms
Op type        Op                  Average runtime (ms)    Pct total runtime
-------------  ----------------  ----------------------  -------------------
call_module    embed_1                       14.1785              47.1456
call_module    unredency_linear               9.78136             32.5244
call_module    mlp_mlp_3                      0.946522             3.14732
call_module    mlp_mlp_6                      0.891447             2.96419
call_module    linear_fc                      0.585794             1.94785
call_module    mlp_mlp_2                      0.469923             1.56256
call_function  add                            0.356674             1.18599
call_module    mlp_mlp_5                      0.317812             1.05677
call_function  add_4                          0.253201             0.841928
call_module    mlp_mlp_8                      0.252247             0.838757
call_module    mlp_mlp_4

# torchscript

In [563]:
def custom_workload_wdl_torchscript(dim,nums_fields,prefix,l = [400,400,400]):
  print(f"now gen workload of wdl with config: dim: {dim}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(nums_fields)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(nums_fields)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_MLP(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  ori = torch.jit.script(ori_traced)
  modify = torch.jit.script(modify_traced)
  ori.eval()
  modify.eval()
  return ori,modify
  # return ori_traced,modify_traced

# dim = 8, 40

In [564]:
ori, modify = custom_workload_wdl_torchscript(8,100,40)

now gen workload of wdl with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [565]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

4.12 ms ± 2.13 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [567]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

3.98 ms ± 963 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [568]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

9.74 ms ± 1.88 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [569]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

9.35 ms ± 1.21 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32,40

In [570]:
ori, modify = custom_workload_wdl_torchscript(32,100,40)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [571]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

5.28 ms ± 2.04 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [572]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

4.65 ms ± 2.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [573]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

20.5 ms ± 2.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [574]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

10.6 ms ± 2.08 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64,40

In [575]:
ori, modify = custom_workload_wdl_torchscript(64,100,40)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [580]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

6.03 ms ± 1.22 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [581]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

4.98 ms ± 1.11 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [584]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

28.7 ms ± 2.14 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [585]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

20 ms ± 1.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128,40

In [586]:
ori, modify = custom_workload_wdl_torchscript(128,100,40)

now gen workload of wdl with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [587]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

15.8 ms ± 3.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [588]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

7.55 ms ± 3.03 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [589]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

48.2 ms ± 2.72 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [590]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

32.6 ms ± 2.71 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [425]:
ori,modify = custom_workload_wdl(64,100,70)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [426]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

6.63 ms ± 1.66 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [427]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long))

5.02 ms ± 1.17 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [428]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

29.3 ms ± 3.12 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [429]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))

11.6 ms ± 1.87 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# custom_workload 50

In [599]:
ori,modify = custom_workload_wdl(32,50,20)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [600]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

4.11 ms ± 878 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [601]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

3.85 ms ± 802 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [604]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

8.46 ms ± 1.69 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [605]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

7.35 ms ± 1.17 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [606]:
ori,modify = custom_workload_wdl(32,50,20,l = [1024,512,256])

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [607]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

5.46 ms ± 1.4 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [608]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

4.78 ms ± 1.02 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [609]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

12.7 ms ± 1.91 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [610]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

10.6 ms ± 1.71 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [591]:
ori,modify = custom_workload_wdl(64,50,20)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [594]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

5.12 ms ± 919 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [595]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

4.53 ms ± 933 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [598]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

19.4 ms ± 2.29 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [597]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

8.9 ms ± 1.89 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [611]:
ori,modify = custom_workload_wdl(64,50,20,l=[1024,512,256])

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [612]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

6.83 ms ± 1.38 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [613]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,50), dtype=torch.long))

5.76 ms ± 1.24 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [614]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

24.4 ms ± 2.35 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [615]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,50), dtype=torch.long))

14.2 ms ± 2.17 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


2

In [231]:
def workload_wdl2(dim):
  print(f"now gen workload of wdl with config: dim: {dim}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(22)],dim,[1024,512,256],0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(22)],dim,[400,400,400],0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_MLP2(modify_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [249]:
ori , modify = workload_wdl2(32)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node offsets target offsets offsets of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [250]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.13 ms ± 1.25 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [251]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.4 ms ± 702 µs per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [254]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

21.4 ms ± 4.78 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [255]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

5.68 ms ± 1.33 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [160]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 4.895687103271484 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_module    mlp_mlp_3                         0.578403             11.8146
call_module    mlp_mlp_6                         0.54884              11.2107
call_module    linear_fc                         0.42057               8.59063
call_module    mlp_mlp_0                         0.409126              8.35687
call_module    mlp_mlp_5                         0.401735              8.2059
call_module    mlp_mlp_2                         0.380516              7.77247
call_module    mlp_mlp_8                         0.347376              7.09555
call_module    embedding_embedding               0.342369              6.99328
call_module    mlp_mlp_9                         0.158548              3.23853
call_module    mlp_mlp_7                         0.146866              2.9999
call_function  add_1 

In [203]:
scripted_model_ori = torch.jit.script(ori)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


In [204]:
input = torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long)

In [212]:
scripted_model_ori.eval()


RecursiveScriptModule(
  original_name=WideAndDeepModel
  (embedding): RecursiveScriptModule(
    original_name=Module
    (embedding): RecursiveScriptModule(original_name=Embedding)
  )
  (linear): RecursiveScriptModule(
    original_name=Module
    (fc): RecursiveScriptModule(original_name=Embedding)
  )
  (mlp): RecursiveScriptModule(
    original_name=Module
    (mlp): RecursiveScriptModule(
      original_name=Module
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=ReLU)
      (2): RecursiveScriptModule(original_name=Dropout)
      (3): RecursiveScriptModule(original_name=Linear)
      (4): RecursiveScriptModule(original_name=ReLU)
      (5): RecursiveScriptModule(original_name=Dropout)
      (6): RecursiveScriptModule(original_name=Linear)
      (7): RecursiveScriptModule(original_name=ReLU)
      (8): RecursiveScriptModule(original_name=Dropout)
      (9): RecursiveScriptModule(original_name=Linear)
    )
  )
)

In [216]:
%timeit -n 10 -r 3 with torch.no_grad(): output = scripted_model_ori(input)

3.07 ms ± 367 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [169]:
sample = torch.ones((4096,22), dtype=torch.long)


torch.onnx.export(ori,sample,f'ori.onnx')

In [207]:
scripted_model = torch.jit.script(modify)


In [208]:
scripted_model.eval()


RecursiveScriptModule(
  original_name=WideAndDeepModel
  (linear): RecursiveScriptModule(
    original_name=Module
    (fc): RecursiveScriptModule(original_name=Embedding)
  )
  (mlp): RecursiveScriptModule(
    original_name=Module
    (mlp): RecursiveScriptModule(
      original_name=Module
      (1): RecursiveScriptModule(original_name=ReLU)
      (2): RecursiveScriptModule(original_name=Dropout)
      (3): RecursiveScriptModule(original_name=Linear)
      (4): RecursiveScriptModule(original_name=ReLU)
      (5): RecursiveScriptModule(original_name=Dropout)
      (6): RecursiveScriptModule(original_name=Linear)
      (7): RecursiveScriptModule(original_name=ReLU)
      (8): RecursiveScriptModule(original_name=Dropout)
      (9): RecursiveScriptModule(original_name=Linear)
    )
  )
  (embed): RecursiveScriptModule(original_name=Embedding)
  (redency_linear): RecursiveScriptModule(original_name=Linear)
  (unredency_linear): RecursiveScriptModule(original_name=Linear)
)

In [209]:
input = torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long)

In [230]:
%timeit -n 10 -r 3 with torch.no_grad(): output = scripted_model( torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

3.28 ms ± 348 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [185]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 14.651298522949219 ms
Op type        Op                  Average runtime (ms)    Pct total runtime
-------------  ----------------  ----------------------  -------------------
call_module    unredency_linear               8.31437             56.7484
call_module    mlp_mlp_2                      0.823259             5.61902
call_module    mlp_mlp_3                      0.820398             5.59949
call_module    mlp_mlp_6                      0.421047             2.87379
call_module    mlp_mlp_5                      0.388145             2.64922
call_module    embed                          0.346899             2.3677
call_module    mlp_mlp_8                      0.329494             2.24891
call_module    linear_fc                      0.293732             2.00482
call_module    mlp_mlp_1                      0.286341             1.95437
call_module    redency_linear                 0.262737             1.79327
call_function  getitem_1                      0.256062          

In [170]:
sample = torch.ones((4096,22), dtype=torch.long)


torch.onnx.export(modify,sample,f'modify.onnx')

# GPU

In [617]:
import time

In [618]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [623]:
ori,modify = custom_workload_wdl(64,100,40)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [624]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

5.4598649342854815

In [625]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

4.596138000488281

In [626]:
sum = 0
ori.to(device)
for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long).to(device)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

30.384627978007

In [627]:
sum = 0
modify.to(device)
for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long).to(device)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

Traceback (most recent call last):
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph_module.py", line 304, in __call__
    return super(self.cls, obj).__call__(*args, **kwargs)  # type: ignore[misc]
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "<eval_with_key>.490 from /home/yssun/pytorch-fm/torchfm/model/test_fx/../wd.py:22 in forward", line 13, in forward
    add_4 = getitem + redency_offset;  getitem = redency_offset = None
RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

Call using an FX-traced Module, line 13 of the traced Module's generated forward function:
    re

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [616]:
ori,modify = custom_workload_wdl(64,100,70)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [621]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

6.074372927347819

In [622]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,100), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

3.7813663482666016

# Frappe

# dim = 8

In [645]:
ori,modify = custom_workload_wdl(8,10,9)

now gen workload of wdl with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [647]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.389383316040039

In [648]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

8.751773834228516

In [649]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

3.392918904622396

In [651]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

12.2283935546875

# dim = 32

In [652]:
ori,modify = custom_workload_wdl(32,10,9)

now gen workload of wdl with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [653]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.416865030924479

In [654]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

3.0974547068277993

In [655]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

3.4563541412353516

In [656]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

4.63411808013916

# dim = 64

In [657]:
ori,modify = custom_workload_wdl(64,10,9)

now gen workload of wdl with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [658]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.5571584701538086

In [659]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.2270520528157554

In [660]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

4.056811332702637

In [661]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.9038031895955405

# dim = 128

In [662]:
ori,modify = custom_workload_wdl(128,10,9)

now gen workload of wdl with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [663]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.9439528783162436

In [664]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(1024,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

2.367417017618815

In [665]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():ori(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

5.027413368225098

In [666]:
sum = 0

for _ in range(30):
  input_data = torch.randint(low=0, high=88, size=(4096,10), dtype=torch.long)
  start = time.time()
  with torch.no_grad():modify(input_data)
  end = time.time()
  sum += (end - start) * 1000
sum / 30

3.300762176513672

In [1]:
from typing import List, Tuple, Union, Any


In [ ]:
class redundancy_shape_info:
  def __init__(self, free_dim: int,
               slice_list: tuple[int | slice | list[int], ...]):
    self.free_dim = free_dim
    self.slice_list = slice_list

In [ ]:
class redundancy_shape_info:
  def __init__(self, orientation_dim: int,
               subscript_list: tuple[list[int], ...]):
    self.orientation_dim = orientation_dim
    self.subscript_list = subscript_list

In [ ]:
redundancy_shape_info(free_dim = 0, slice_list = (0, slice(None,2)))

In [ ]:
redundancy_shape_info(free_dim = 0, slice_list = [3, 5])

In [ ]:
redundancy_shape_info(free_dim = 0, slice_list = [0, 1, 3, 5])